In [16]:
import pandas as pd
import numpy as np

In [23]:
cdc2023 = pd.read_csv(r"data_files\CDC_2023.csv")

In [24]:
cdc2023.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,DROCDY4_,_RFBING6,_DRNKWK2,_RFDRHV8,_FLSHOT7,_PNEUMO3,_AIDTST4,_RFSEAT2,_RFSEAT3,_DRNKDRV
0,1.0,1.0,b'03012023',b'03',b'01',b'2023',1100.0,b'2023000001',2.023000e+09,1.0,...,5.397605e-79,1.0,5.397605e-79,1.0,2.0,2.0,2.0,1.0,1.0,9.0
1,1.0,1.0,b'01062023',b'01',b'06',b'2023',1100.0,b'2023000002',2.023000e+09,1.0,...,5.397605e-79,1.0,5.397605e-79,1.0,1.0,1.0,2.0,1.0,1.0,9.0
2,1.0,1.0,b'03082023',b'03',b'08',b'2023',1100.0,b'2023000003',2.023000e+09,1.0,...,5.397605e-79,1.0,5.397605e-79,1.0,1.0,1.0,2.0,1.0,1.0,9.0
3,1.0,1.0,b'03062023',b'03',b'06',b'2023',1100.0,b'2023000004',2.023000e+09,1.0,...,5.397605e-79,1.0,5.397605e-79,1.0,1.0,1.0,1.0,1.0,1.0,9.0
4,1.0,1.0,b'01062023',b'01',b'06',b'2023',1100.0,b'2023000005',2.023000e+09,1.0,...,7.000000e+00,1.0,4.700000e+01,1.0,2.0,1.0,2.0,1.0,1.0,2.0


In [25]:
new_list = [
    "IMONTH", "IYEAR", "DISPCODE", "CADULT1", "SEXVAR", "PVTRESD3",
    "GENHLTH", "PHYSHLTH", "MENTHLTH", "PRIMINS1", "MEDCOST1", "CHECKUP1",
    "EXERANY2", "EXRACT12", "BPHIGH6", "BPMEDS1", "CHOLCHK3", "TOLDHI3",
    "CHOLMED3", "CVDINFR4", "CVDCRHD4", "CVDSTRK3", "ASTHMA3", "ASTHNOW",
    "CHCCOPD3", "CHCSCNC1", "CHCKDNY2", "HAVARTH4", "DIABETE4", "DIABAGE4",
    "MARITAL", "EDUCA", "CHILDREN", "INCOME3", "PREGNANT", "WEIGHT2",
    "HEIGHT3", "DECIDE", "DEAF", "SMOKE100", "SMOKDAY2", "USENOW3",
    "ECIGNOW2", "ALCDAY4", "AVEDRNK3", "DRNK3GE5", "HIVTST7", "PDIABTS1",
    "PREDIAB2", "DIABTYPE", "INSULIN1", "CHKHEMO3", "EYEEXAM1", "DIABEYE1",
    "FEETSORE", "STOPSMK2", "HASYMP1", "HASYMP2", "HASYMP3", "HASYMP4",
    "HASYMP5", "HASYMP6", "STRSYMP1", "STRSYMP2", "STRSYMP3", "STRSYMP4",
    "STRSYMP5", "STRSYMP6", "ACEDRINK", "SDLONELY", "_BMI5CAT"
]


In [26]:
cdc2023 = cdc2023[new_list]

In [27]:
cdc2023.tail()

,IMONTH,IYEAR,DISPCODE,CADULT1,SEXVAR,PVTRESD3,GENHLTH,PHYSHLTH,MENTHLTH,PRIMINS1,...,HASYMP6,STRSYMP1,STRSYMP2,STRSYMP3,STRSYMP4,STRSYMP5,STRSYMP6,ACEDRINK,SDLONELY,_BMI5CAT
433318,b'12',b'2023',1100.0,1.0,1.0,1.0,3.0,12.0,30.0,77.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
433319,b'01',b'2024',1100.0,1.0,2.0,1.0,2.0,88.0,88.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
433320,b'12',b'2023',1100.0,1.0,2.0,1.0,2.0,10.0,88.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
433321,b'12',b'2023',1100.0,1.0,2.0,1.0,3.0,88.0,88.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
433322,b'12',b'2023',1100.0,1.0,1.0,1.0,5.0,2.0,88.0,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [28]:
cdc2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433323 entries, 0 to 433322
Data columns (total 71 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   IMONTH    433323 non-null  object 
 1   IYEAR     433323 non-null  object 
 2   DISPCODE  433323 non-null  float64
 3   CADULT1   344978 non-null  float64
 4   SEXVAR    433323 non-null  float64
 5   PVTRESD3  344978 non-null  float64
 6   GENHLTH   433319 non-null  float64
 7   PHYSHLTH  433320 non-null  float64
 8   MENTHLTH  433320 non-null  float64
 9   PRIMINS1  433318 non-null  float64
 10  MEDCOST1  433321 non-null  float64
 11  CHECKUP1  433321 non-null  float64
 12  EXERANY2  433321 non-null  float64
 13  EXRACT12  325227 non-null  float64
 14  BPHIGH6   433320 non-null  float64
 15  BPMEDS1   176222 non-null  float64
 16  CHOLCHK3  433320 non-null  float64
 17  TOLDHI3   381511 non-null  float64
 18  CHOLMED3  379883 non-null  float64
 19  CVDINFR4  433320 non-null  float64
 20  CVDC

In [29]:
cdc2023.describe()

,DISPCODE,CADULT1,SEXVAR,PVTRESD3,GENHLTH,PHYSHLTH,MENTHLTH,PRIMINS1,MEDCOST1,CHECKUP1,...,HASYMP6,STRSYMP1,STRSYMP2,STRSYMP3,STRSYMP4,STRSYMP5,STRSYMP6,ACEDRINK,SDLONELY,_BMI5CAT
count,433323.000000,344978.0,433323.000000,344978.000000,433319.000000,433320.000000,433320.000000,433318.000000,433321.000000,433321.000000,...,10475.000000,10438.000000,10424.000000,10416.000000,10403.000000,10391.000000,10375.000000,55939.000000,225998.000000,392788.000000
mean,1117.783963,1.0,1.529723,1.003913,2.629645,58.477843,58.022037,10.865293,1.933550,1.434567,...,1.517136,1.317973,1.199156,2.232239,2.711237,1.525551,2.284819,1.855289,4.042461,2.999674
std,38.237812,0.0,0.499116,0.062434,1.081803,37.734401,37.989494,24.899418,0.436001,1.136273,...,1.522088,1.301882,1.037369,2.318748,2.329941,1.617903,2.290948,0.915790,1.050912,0.834410
min,1100.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1100.000000,1.0,1.000000,1.000000,2.000000,14.000000,10.000000,1.000000,2.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,2.000000
50%,1100.000000,1.0,2.000000,1.000000,3.000000,88.000000,88.000000,3.000000,2.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,2.000000,4.000000,3.000000
75%,1100.000000,1.0,2.000000,1.000000,3.000000,88.000000,88.000000,5.000000,2.000000,1.000000,...,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,2.000000,2.000000,5.000000,4.000000
max,1200.000000,1.0,2.000000,2.000000,9.000000,99.000000,99.000000,99.000000,9.000000,9.000000,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,4.000000


In [30]:
df_duplicates = cdc2023.duplicated()
print("Duplicate rows in data:")
print(df_duplicates.value_counts())

Duplicate rows in data:
False    433306
True         17
Name: count, dtype: int64


In [31]:
df_duplicates_rows = cdc2023[cdc2023.duplicated(keep=False)]

df_duplicates_rows

,IMONTH,IYEAR,DISPCODE,CADULT1,SEXVAR,PVTRESD3,GENHLTH,PHYSHLTH,MENTHLTH,PRIMINS1,...,HASYMP6,STRSYMP1,STRSYMP2,STRSYMP3,STRSYMP4,STRSYMP5,STRSYMP6,ACEDRINK,SDLONELY,_BMI5CAT
31948,b'05',b'2023',1100.0,1.0,1.0,1.0,9.0,99.0,99.0,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN
32008,b'05',b'2023',1100.0,1.0,1.0,1.0,9.0,99.0,99.0,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN
32321,b'05',b'2023',1100.0,1.0,1.0,1.0,9.0,99.0,99.0,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN
42134,b'06',b'2023',1200.0,1.0,1.0,1.0,3.0,88.0,88.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42211,b'06',b'2023',1200.0,1.0,2.0,1.0,3.0,88.0,88.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62716,b'03',b'2023',1200.0,1.0,2.0,1.0,1.0,88.0,88.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69284,b'02',b'2023',1200.0,1.0,2.0,1.0,1.0,88.0,88.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102268,b'04',b'2023',1200.0,1.0,2.0,1.0,1.0,88.0,88.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103397,b'08',b'2023',1200.0,1.0,1.0,1.0,1.0,88.0,88.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113309,b'08',b'2023',1200.0,1.0,1.0,1.0,1.0,88.0,88.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
cdc2023.drop_duplicates(inplace=True)

print('Duplicates on data removed. Number of rows remaining:', cdc2023.shape[0])

Duplicates on data removed. Number of rows remaining: 433306


In [33]:
# renaming columns for training and test datasets
cdc2023.rename(columns={"IMONTH" :"interview_month",
                         "IYEAR" : "interview_year",
                          "DISPCODE" : "final_dispcode", 
                          "CADULT1" : "cell_adult1",
                          "SEXVAR" : "sex_var",
                          "PVTRESD3" : "private_residence3",
                          "GENHLTH" : "general_health", 
                          "PHYSHLTH" : "phys_health",
                          "MENTHLTH" : "mental_health",
                          "PRIMINS1" : "prim_insurance",
                          "MEDCOST1" : "med_cost",
                          "CHECKUP1" : "checkup1",
                          "EXERANY2" : "exer_any2",
                          "EXRACT12" : "phycial_act12",
                          "BPHIGH6" : "bp_high", 
                          "BPMEDS1" : "bp_meds1",
                          "CHOLCHK3" : "chol_check3",
                          "TOLDHI3" : "told_chol_high3",
                          "CHOLMED3" : "chol_med3",
                          "CVDINFR4" : "diagnosed_heart_attack4",
                          "CVDCRHD4" : "diagnosed_angina_or_cor_heart",
                          "CVDSTRK3" : "diagosed_stroke3", 
                          "ASTHMA3" : "asthma3", 
                          "ASTHNOW" : "asth_now",
                          "CHCCOPD3" : "copd_emphysema_or_chronic_bronchitis",
                          "CHCSCNC1" : "skin_cancer_not_melanoma", 
                          "CHCKDNY2" : "told_kidney_disease",
                          "HAVARTH4" : "told_arthritis",
                          "DIABETE4" : "diabetes4", 
                          "DIABAGE4" : "age_diabetes4",
                          "MARITAL" : "marital", 
                          "EDUCA" : "education", 
                          "CHILDREN" : "children",
                          "INCOME3" : "income3", 
                          "PREGNANT" : "pregnancy",
                          "WEIGHT2" : "weight_pounds",
                          "HEIGHT3" : "height_feet_inches", 
                          "DECIDE" : "difficulty_concentrating_or_remembering", 
                          "DEAF" : "deaf_or_diffi_hearing",
                          "SMOKE100" : "smoked_100cigarettes", 
                          "SMOKDAY2" : "smokeday2",
                          "USENOW3" : "smokeless_tobacco",
                          "ECIGNOW2": "e-cigars_or_vaping",
                          "ALCDAY4" : "alco_days", 
                          "AVEDRNK3": "avg_alco_per_day", 
                          "DRNK3GE5" : "binge_drinking",
                          "HIVTST7" : "ever_tested_HIV", 
                          "PDIABTS1" : "when_blood_test ",
                          "PREDIAB2" : "prediabetes_or_borderline_diabetes",
                          "DIABTYPE" : "diabetes_type",
                          "INSULIN1" : "insulin", 
                          "CHKHEMO3" : "times_Checked_glyco_hemoglobin",
                          "EYEEXAM1" : "eye_exam_pupil_dilated", 
                          "DIABEYE1" : "back_eye_photo",
                          "FEETSORE" : "feet_sores_irritation",
                          "STOPSMK2" : "no_smoking_in_12months",
                          "HASYMP1"  : "heart_attack_symptoms_neck",
                          "HASYMP2" : "heart_attack_symptoms_faint", 
                          "HASYMP3" : "heart_attack_symptoms_chest", 
                          "HASYMP4" : "heart_attack_symptoms_vision",
                          "HASYMP5" : "heart_attack_symptoms_arms",
                          "HASYMP6" : "heart_attack_symptoms_breath_shortness",
                          "STRSYMP1" : "stroke_symptoms_confusion",
                          "STRSYMP2" : "stroke_symtoms_numbness", 
                          "STRSYMP3" : "stroke_symptoms_vision",
                          "STRSYMP4" : "stroke_symptoms_chest",
                          "STRSYMP5" : "stroke_symptoms_dizziness",
                          "STRSYMP6" : "stroke_symptoms_headache",
                          "ACEDRINK" : "live_with_alcoholic", 
                          "SDLONELY" : "feel_lonely", 
                          "_BMI5CAT" : "bmi_categories"},
                inplace= True)

In [34]:
cdc2023.head()

,interview_month,interview_year,final_dispcode,cell_adult1,sex_var,private_residence3,general_health,phys_health,mental_health,prim_insurance,...,heart_attack_symptoms_breath_shortness,stroke_symptoms_confusion,stroke_symtoms_numbness,stroke_symptoms_vision,stroke_symptoms_chest,stroke_symptoms_dizziness,stroke_symptoms_headache,live_with_alcoholic,feel_lonely,bmi_categories
0,b'03',b'2023',1100.0,NaN,2.0,NaN,2.0,88.0,88.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0
1,b'01',b'2023',1100.0,NaN,2.0,NaN,2.0,88.0,88.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0
2,b'03',b'2023',1100.0,NaN,2.0,NaN,4.0,6.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0
3,b'03',b'2023',1100.0,NaN,2.0,NaN,2.0,2.0,88.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0
4,b'01',b'2023',1100.0,NaN,2.0,NaN,4.0,88.0,88.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0
